In [ ]:
# !pip install sqlalchemy pandas
# !pip install pandasql

In [ ]:
import pandas as pd
import pandasql as psql

In [ ]:
# Load the CSV into a DataFrame
fact = pd.read_csv('fact.csv')
# orders = pd.read_csv(orders_path)
# products = pd.read_csv(products_path)
# states = pd.read_csv(states_path)
# time = pd.read_csv(time_path)
# users = pd.read_csv(users_path)

In [ ]:
fact.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,quantity,user_id,time_id,state_id
0,0,2,33120,1,1,3,202279,129,13
1,1,2,28985,2,1,4,202279,129,13
2,2,2,9327,3,0,2,202279,129,13
3,3,2,45918,4,1,4,202279,129,13
4,4,2,30035,5,0,1,202279,129,13


In [ ]:
orders = pd.read_csv('orders.csv')
orders.head()

,Unnamed: 0,order_id,order_number,days_since_prior_order,Revenue
0,0,1,4,9.0,153.772904
1,1,2,3,8.0,169.975928
2,2,3,16,12.0,123.059261
3,3,4,36,7.0,140.697503
4,4,5,42,9.0,471.582088


In [ ]:
products = pd.read_csv('products.csv')
products.head()

,Unnamed: 0,product_id,product_name,unit_price,aisle,department
0,0,1,Chocolate Sandwich Cookies,4.236623,cookies cakes,snacks
1,1,2,All-Seasons Salt,8.648061,spices seasonings,pantry
2,2,3,Robust Golden Unsweetened Oolong Tea,7.973975,tea,beverages
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,7.674953,frozen meals,frozen
4,4,5,Green Chile Anytime Sauce,3.662800,marinades meat preparation,pantry


In [ ]:
states = pd.read_csv('states.csv')
states.head()

,Unnamed: 0,state,state_id
0,0,Alabama,1
1,1,Alaska,2
2,2,Arizona,3
3,3,Arkansas,4
4,4,California,5


In [ ]:
users = pd.read_csv('users.csv')
users.head()

,Unnamed: 0,user_id
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5


In [ ]:
time = pd.read_csv('time.csv')
time.head()

,Unnamed: 0,dow,hour_of_day,time_indx
0,0,0,0,0
1,1,0,1,1
2,2,0,2,2
3,3,0,3,3
4,4,0,4,4


In [ ]:
# Initialize pandasql
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
# Calculate total revenue per product category using CTE
query = """
WITH ProductCategoryRevenue AS (
    SELECT
        p.aisle,
        SUM(f.quantity * p.unit_price) AS total_revenue
    FROM
        fact f
    JOIN
        products p ON f.product_id = p.product_id
    GROUP BY
        p.aisle
)
SELECT
    aisle,
    total_revenue
FROM
    ProductCategoryRevenue
ORDER BY
    total_revenue DESC;
"""

revenue_per_category = pysqldf(query)
print(revenue_per_category)

MemoryError: 

In [ ]:
def process_chunk(fact, query):
    time = pd.read_csv('time.csv')
    products = pd.read_csv('products.csv')
    orders = pd.read_csv('orders.csv')
    users = pd.read_csv('users.csv')
    states = pd.read_csv('states.csv')
    result = psql.sqldf(query, locals())

    return result


## How to identify states contributing the most to the revenue?  
California leads in total revenue, and that may be due to its large population, while Wyoming contributes the least, which can be attributed to its smaller population size. That is why we need to investigate the population size corresponding to every state.


In [ ]:
chunk_size = 1000000  # Adjust chunk size based on your system's memory
fact_chunks = pd.read_csv("fact.csv", chunksize=chunk_size)
chunk = next(fact_chunks)

# Create an empty DataFrame to store intermediate results
aisle_revenue = pd.DataFrame()

# Process each chunk
# for chunk in fact_chunks:
    # Process the chunk and get revenue per aisle
query = """
    SELECT
        s.state,
        SUM(o.revenue) AS total_revenue,
        RANK() OVER (ORDER BY SUM(o.revenue) DESC) AS state_rank
    FROM
        orders o
    JOIN
        fact f ON o.order_id = f.order_id
    JOIN
        states s ON f.state_id = s.state_id
    GROUP BY
        s.state
    ORDER BY
        total_revenue DESC;
"""
chunk_result = process_chunk(chunk, query)

display(chunk_result)


,state,total_revenue,state_rank
0,California,3.091068e+07,1
1,Texas,2.402438e+07,2
2,Florida,1.787497e+07,3
3,New York,1.631207e+07,4
4,Illinois,1.033478e+07,5
5,Pennsylvania,1.009320e+07,6
6,Ohio,9.484065e+06,7
7,North Carolina,8.368202e+06,8
8,Michigan,8.184565e+06,9
9,Georgia,8.025056e+06,10


## How to identify Customers who reorder specific products frequently ?

The "Bag of Bananas" (Product ID 13176) is the most popular and frequently reordered product, indicating strong, consistent demand and customer loyalty. The maximum of 7 reorders per customer suggests regular consumption, highlighting the product's appeal and the potential for targeted marketing, inventory planning, and pricing strategies to maintain customer satisfaction and optimize sales.




In [ ]:
query = """
    SELECT
        f.user_id,
        f.product_id,
        COUNT(f.reordered) AS reorder_count,
        RANK() OVER (PARTITION BY f.product_id ORDER BY COUNT(f.reordered) DESC) AS
    customer_rank
    FROM
        fact f
    WHERE
        f.reordered = 1
    GROUP BY
        f.user_id, f.product_id
    ORDER BY
        reorder_count DESC;
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result[chunk_result['product_id'] == 13176])

,user_id,product_id,reorder_count,customer_rank
8,106826,13176,7,1
27,14868,13176,6,2
28,19528,13176,6,2
29,55695,13176,6,2
30,84474,13176,6,2
...,...,...,...,...
169587,206062,13176,1,1550
169588,206071,13176,1,1550
169589,206104,13176,1,1550
169590,206193,13176,1,1550


In [ ]:
products[products['product_id'] == 13176]

,Unnamed: 0,product_id,product_name,unit_price,aisle,department
13175,13175,13176,Bag of Organic Bananas,6.389297,fresh fruits,produce


## Understand product demand by time of day and department:
This gives insights into the product demand by time of day and department, revealing peak sales hours for each product within its department. By ranking products based on quantity sold per hour, it helps identify the times when products are most popular, enabling better inventory management, targeted promotions, and optimized staffing.

In [ ]:
query = """
    SELECT
        t.hour_of_day,
        p.department,
        SUM(f.quantity) AS total_quantity_sold,
        ROW_NUMBER() OVER (PARTITION BY p.department ORDER BY SUM(f.quantity) DESC) AS hour_rank
    FROM
        fact f
    JOIN
        time t ON f.time_id = t.time_indx
    JOIN
        products p ON f.product_id = p.product_id
    GROUP BY
        t.hour_of_day, p.department
    ORDER BY
        p.department, hour_rank;
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,hour_of_day,department,total_quantity_sold,hour_rank
0,16,alcohol,1173,1
1,12,alcohol,1130,2
2,15,alcohol,1082,3
3,14,alcohol,1059,4
4,11,alcohol,1040,5
...,...,...,...,...
499,1,snacks,1055,20
500,5,snacks,613,21
501,3,snacks,481,22
502,2,snacks,435,23


## Reorder rate by product:
Calculating the reorder rate for each product, which measures the proportion of orders for a product that were reordered. Products with a high reorder rate likely have strong customer loyalty and are more frequently purchased by customers who return to buy them again. While products with a low reorder rate indicate weaker customer retention or a lack of repeat demand, suggesting that these products may not be as essential or consumable to customers.

In [ ]:
query = """
    SELECT
        p.product_name,
        COUNT(CASE WHEN f.reordered = 1 THEN 1 END) * 1.0 / COUNT(f.product_id) AS reorder_rate
    FROM
        fact f
    JOIN
        products p ON f.product_id = p.product_id
    GROUP BY
        p.product_name
    ORDER BY
        reorder_rate DESC;
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,product_name,reorder_rate
0,with a Splash of Mango Coconut Water,1.0
1,with Xylitol Watermelon Twist 18 Sticks Sugar ...,1.0
2,with Xylitol Minty Sweet Twist 18 Sticks Sugar...,1.0
3,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,1.0
4,with Seasoned Roasted Potatoes Scrambled Eggs ...,1.0
...,...,...
35093,100 Calorie Right Bites Cheez It Snack Mix,0.0
35094,100 Calorie Healthy Pop Butter Microwave Pop Corn,0.0
35095,"1,000 Mg Vitamin C Orange Effervescent Powdere...",0.0
35096,1% Low Fat Cottage Cheese,0.0


In [ ]:
chunk_result[(chunk_result['reorder_rate'] < 1) & (chunk_result['reorder_rate'] > 0.0001)]

,product_name,reorder_rate
3967,Cranberry Pomegranate Sparkling Yerba Mate,0.944444
3968,Yerba Mate Orange Exuberance Tea,0.941176
3969,Dairy Lemonade,0.941176
3970,"Coconut Water, Real",0.937500
3971,Original Vanilla Nutrition Shake,0.933333
...,...,...
25664,Peppercorn Medley Grinder,0.035714
25665,Champagne Vinegar,0.035714
25666,Oregano Leaves,0.034483
25667,Garlic Salt,0.034483


## Find the top users based on their total revenue
Identify which users contribute the most to your revenue (e.g., top 10 or 20 users by rank). These high-value customers can be your primary targets for retention and loyalty programs. We can Offer them special discounts on the most used products , personalized offers, or rewards to these users to maintain and grow their engagement.

In [ ]:
query = """
    SELECT
        f.user_id,
        SUM(o.revenue) AS total_revenue,
        RANK() OVER (ORDER BY SUM(o.revenue) DESC) AS user_rank
    FROM
        fact f
    JOIN
        orders o ON o.order_id = f.order_id
    GROUP BY
        f.user_id
    ORDER BY
        user_rank;

    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,user_id,total_revenue,user_rank
0,60694,233060.971499,1
1,22906,229951.659444,2
2,2335,223226.233227,3
3,72377,194165.295048,4
4,201268,192758.390789,5
...,...,...,...
67031,114905,1.224860,67032
67032,165936,0.000000,67033
67033,104845,0.000000,67033
67034,97384,0.000000,67033


##  Find the top users based on their total revenue and the top products for each one of these users


In [ ]:
query = """
    WITH UserRevenue AS (
        SELECT
            f.user_id,
            SUM(o.revenue) AS total_revenue,
            RANK() OVER (ORDER BY SUM(o.revenue) DESC) AS user_rank
        FROM
            fact f
        JOIN
            orders o ON f.order_id = o.order_id
        GROUP BY
            f.user_id
    ),
    TopUsers AS (
        SELECT
            user_id,
            total_revenue
        FROM
            UserRevenue
        WHERE
            user_rank <= 10  -- Change this value to get more or fewer top users
    ),
    TopProducts AS (
        SELECT
            f.user_id,
            p.product_name,
            SUM(f.quantity) AS total_quantity,
            ROW_NUMBER() OVER (PARTITION BY f.user_id ORDER BY SUM(f.quantity) DESC) AS product_rank
        FROM
            fact f
        JOIN
            products p ON f.product_id = p.product_id
        WHERE
            f.user_id IN (SELECT user_id FROM TopUsers)
        GROUP BY
            f.user_id, p.product_name
    )
    SELECT
        t.user_id,
        u.total_revenue,
        t.product_name,
        t.total_quantity,
        t.product_rank
    FROM
        TopProducts t
    JOIN
        TopUsers u ON t.user_id = u.user_id
    WHERE
        t.product_rank <= 3 -- Top 3 products for each user
    ORDER BY
        u.total_revenue DESC, t.user_id, t.product_rank;

    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,user_id,total_revenue,product_name,total_quantity,product_rank
0,60694,233060.971499,Lemon Yogurt,32,1
1,60694,233060.971499,Honey Yoghurt,24,2
2,60694,233060.971499,Raspberry Yoghurt,20,3
3,22906,229951.659444,Total Greek Strained Yogurt,9,1
4,22906,229951.659444,Cream Top Smooth & Creamy Vanilla Yogurt,9,2
5,22906,229951.659444,PlusShot Strawberry Probiotic Juice Drink,8,3
6,2335,223226.233227,Organic Whole Milk Strawberry Beet Berry Yogur...,20,1
7,2335,223226.233227,Mini Round Containers + Lids,15,2
8,2335,223226.233227,Banana,15,3
9,72377,194165.295048,Almond Nut & Rice Cracker Snacks,10,1


## Sales Distribution by Time (Day of the Week):
Identify which days generate the most or least revenue. For example, weekends may see higher revenue due to increased shopping activity, while weekdays might show more consistent patterns. So We can focus marketing campaigns or promotions on high-revenue days to maximize returns and boost engagement on low-revenue days with discounts, special offers, or flash sales.

In [ ]:
query = """
    SELECT t.dow, SUM(o.Revenue) AS total_revenue
    FROM orders o
    JOIN fact f ON f.order_id = o.order_id
    JOIN time t ON t.time_indx = f.time_id
    GROUP BY t.dow
    ORDER BY t.dow;

    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,dow,total_revenue
0,0,5.293017e+07
1,1,4.613861e+07
2,2,3.441187e+07
3,3,3.020874e+07
4,4,2.906277e+07
5,5,3.388003e+07
6,6,3.854416e+07


## Product Popularity (Top Products Ordered By Category)


In [ ]:
query = """
    WITH ProductPopularity AS (
        SELECT
            p.product_name,
            p.department,
            SUM(f.quantity) AS total_quantity_ordered,
            COUNT(DISTINCT f.order_id) AS total_orders,
            RANK() OVER (PARTITION BY p.department ORDER BY SUM(f.quantity) DESC) AS department_rank
        FROM
            fact f
        JOIN
            products p ON f.product_id = p.product_id
        GROUP BY
            p.product_name, p.department
    )
    SELECT
        product_name,
        department,
        total_quantity_ordered,
        total_orders,
        department_rank
    FROM
        ProductPopularity
    WHERE
        department_rank <= 3 -- Top 3 products per department
    ORDER BY
        department, department_rank;

    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,product_name,department,total_quantity_ordered,total_orders,department_rank
0,Sauvignon Blanc,alcohol,1173,239,1
1,Chardonnay,alcohol,938,183,2
2,Pinot Grigio,alcohol,526,105,3
3,Spinach Peas & Pear Stage 2 Baby Food,babies,1341,264,1
4,Baby Food Stage 2 Blueberry Pear & Purple Carrot,babies,1314,265,2
...,...,...,...,...,...
58,Bag of Organic Bananas,produce,59495,11867,2
59,Organic Strawberries,produce,40506,8138,3
60,Hint Of Sea Salt Almond Nut Thins,snacks,2827,380,1
61,Crackers Cheddar Bunnies Snack Packs,snacks,2794,372,2


## Top 3 Products by Revenue in Each Department:

In [ ]:
query = """
    SELECT
        sub.department,
        sub.product_name,
        sub.total_revenue,
        sub.product_rank
    FROM
        (SELECT
            p.department,
            p.product_name,
            SUM(o.revenue) AS total_revenue,
            RANK() OVER (PARTITION BY p.department ORDER BY SUM(o.revenue) DESC) AS product_rank
        FROM
            fact f
        JOIN
            products p ON f.product_id = p.product_id
        JOIN
            orders o ON f.order_id = o.order_id
        GROUP BY
            p.department, p.product_name) sub
    WHERE
        sub.product_rank <= 3
    ORDER BY
        sub.department, sub.product_rank;
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,department,product_name,total_revenue,product_rank
0,alcohol,Sauvignon Blanc,7.791361e+04,1
1,alcohol,Cabernet Sauvignon,5.468394e+04,2
2,alcohol,Chardonnay,4.650841e+04,3
3,babies,Spinach Peas & Pear Stage 2 Baby Food,1.848905e+05,1
4,babies,Baby Food Stage 2 Blueberry Pear & Purple Carrot,1.673981e+05,2
...,...,...,...,...
58,produce,Bag of Organic Bananas,2.876082e+06,2
59,produce,Organic Strawberries,2.261593e+06,3
60,snacks,Lightly Salted Baked Snap Pea Crisps,2.224662e+05,1
61,snacks,Original Veggie Straws,1.606993e+05,2


**Popular Products:** Weekly deals or items sold at near cost, like discounted soft drinks or bulk items.

**Profitable Products:** Private-label brands, impulse purchases at checkout (e.g., candy, gum), or specialty products.

***Balancing Both***
Businesses often aim to strike a balance where their popular products drive traffic and volume while profitable products maximize earnings.
A common strategy is to use popular products as entry points and promote profitable products as add-ons or premium options.

Example: Milk and Dairy
Popular Product: Milk is a grocery staple, sold in high quantities at a low margin. It's often placed at the back of the store to encourage customers to pass other items.
Profitable Add-On: Premium dairy products like organic milk, plant-based alternatives (e.g., almond or oat milk), or specialty cheeses have higher margins and are strategically displayed nearby.

## Product Popularity (Top Products Ordered) By State
Again, promotions and sales in the specific states , also accommodating demand.
Determine which products perform best in each state. So we can focus marketing efforts, inventory allocation, or promotions on the most popular products in each region. Also understand how demand for products varies across states. For example, some states might have higher demand for specific products due to local preferences, climate, or demographics ,so we can tailor product offerings for specific regions based on their preferences and expand advertising efforts for high-demand products in underperforming states.
Identify regions with high product demand to avoid stockouts and reduce surplus inventory in low-demand areas.


In [ ]:
query = """
    SELECT p.product_name, SUM(f.quantity)  AS total_quantity , s.state
    FROM fact f
    JOIN products p ON f.product_id = p.product_id
    JOIN states s ON f.state_id = s.state_id
    GROUP BY p.product_name , s.state
    ORDER BY total_quantity DESC;
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,product_name,total_quantity,state
0,Banana,8359,California
1,Bag of Organic Bananas,7011,California
2,Banana,6455,Texas
3,Bag of Organic Bananas,5284,Texas
4,Banana,5143,Florida
...,...,...,...
300858,with Xylitol Original Flavor 18 Sticks Sugar F...,1,Colorado
300859,with Xylitol Original Flavor 18 Sticks Sugar F...,1,Ohio
300860,with Xylitol Original Flavor 18 Sticks Sugar F...,1,Wisconsin
300861,with a Splash of Mango Coconut Water,1,South Carolina


## Counting Total Number of Orders For Each Customer
### Insight: This helps identify loyal customers who can be targeted for loyalty programs or personalized offers.

In [ ]:
query = """
    SELECT user_id, COUNT(DISTINCT order_id) AS total_orders
    FROM fact
    GROUP BY user_id
    HAVING COUNT(DISTINCT order_id) > 1
    ORDER BY total_orders DESC;
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,user_id,total_orders
0,34340,13
1,140395,11
2,201248,10
3,174555,10
4,169586,10
...,...,...
20812,63,2
20813,42,2
20814,27,2
20815,21,2


# The Average Order Size
###### Insight: Understanding the average order size can help optimize inventory

In [ ]:
query = """
    SELECT AVG(order_size) AS avg_order_size
    FROM (
        SELECT order_id, COUNT(product_id) AS order_size
        FROM fact
        GROUP BY order_id
    ) AS order_sizes;
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,avg_order_size
0,10.074552


### Calculate the percentage of products that are reordered (reordered = 1).


In [ ]:
query = """
    SELECT
        (SUM(reordered) * 1.0 / COUNT(*)) * 100 AS reorder_rate
    FROM fact;
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,reorder_rate
0,59.0268


## Get the Highest and Least selling product within each category based on quantity sold
The results provide insights into which products are performing well and which are underperforming, within each category helping with inventory management and marketing strategies.
By analyzing this data, businesses can optimize product offerings,and focus efforts on promoting popular items or reduce the production of underperforming ones.

In [ ]:
query = """
    SELECT
        product_name,
        department,
        total_quantity,
        product_type
    FROM (
        SELECT
            p.product_name,
            p.department,
            SUM(f.quantity) AS total_quantity,
            RANK() OVER (PARTITION BY p.department ORDER BY SUM(f.quantity) DESC) AS top_rank,
            RANK() OVER (PARTITION BY p.department ORDER BY SUM(f.quantity) ASC) AS least_rank,
            CASE
                WHEN RANK() OVER (PARTITION BY p.department ORDER BY SUM(f.quantity) DESC) = 1 THEN 'Top Selling'
                WHEN RANK() OVER (PARTITION BY p.department ORDER BY SUM(f.quantity) ASC) = 1 THEN 'Least Selling'
            END AS product_type
        FROM
            fact f
        JOIN
            products p ON f.product_id = p.product_id
        GROUP BY
            p.product_name, p.department
    ) ranked_products
    WHERE
        top_rank = 1 OR least_rank = 1
    ORDER BY
        department;

    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,product_name,department,total_quantity,product_type
0,Sauvignon Blanc,alcohol,1173,Top Selling
1,16 Year Single Malt Scotch Whisky,alcohol,1,Least Selling
2,312 Urban Wheat,alcohol,1,Least Selling
3,Ale,alcohol,1,Least Selling
4,Ale Premier,alcohol,1,Least Selling
...,...,...,...,...
2779,Wicked Dark 95% Chocolate,snacks,1,Least Selling
2780,Wintermint Sugar-Free Gum,snacks,1,Least Selling
2781,Wrigley's 5 React2 Unique Mint Sugar-Free Gum,snacks,1,Least Selling
2782,XL Milk Chocolate Peanut Butter Candy Bar,snacks,1,Least Selling


## To find most sold product pairs within each category/ find patterns in reorganizing the store
The output reveals the most frequently sold product pairs within each department, such as the top-selling combinations like India Pale Ale and Beer in the alcohol department, or Cabernet Sauvignon and Chardonnay, indicating that customers often purchase these items together. This insight is valuable for reorganizing the store or website by putting complementary products near each other to facilitate easy cross-selling and increase sales. For example, alcohol-related products like Sauvignon Blanc and Chardonnay could be grouped together, while snacks like Wavy Ranch Potato Chips and Beef Summer Sausage can be placed close to enhance customer experience. Additionally, pairing products with lower sales frequency, such as Zero Non-Chlorine Bleach with 75% Dark Chocolate Bar, can inspire new promotional strategies or bundle offers to boost sales of lesser-known items.


In [ ]:
query = """
    WITH ProductPairs AS (
        SELECT
            f1.product_id AS product_id_1,
            f2.product_id AS product_id_2,
            p1.department,
            COUNT(*) AS pair_count
        FROM
            fact f1
        JOIN
            fact f2 ON f1.order_id = f2.order_id
        JOIN
            products p1 ON f1.product_id = p1.product_id
        JOIN
            products p2 ON f2.product_id = p2.product_id
        WHERE
            f1.product_id < f2.product_id  -- To avoid duplicate pairs
        GROUP BY
            f1.product_id, f2.product_id, p1.department
    )
    SELECT
        p1.product_name AS product_1,
        p2.product_name AS product_2,
        pp.department,
        pp.pair_count
    FROM
        ProductPairs pp
    JOIN
        products p1 ON pp.product_id_1 = p1.product_id
    JOIN
        products p2 ON pp.product_id_2 = p2.product_id
    ORDER BY
        pp.department, pp.pair_count DESC;


    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,product_1,product_2,department,pair_count
0,India Pale Ale,Beer,alcohol,30
1,Cabernet Sauvignon,Chardonnay,alcohol,20
2,Sauvignon Blanc,Chardonnay,alcohol,18
3,Pinot Noir,Cabernet Sauvignon,alcohol,18
4,Sauvignon Blanc,Banana,alcohol,16
...,...,...,...,...
4048214,75% Dark Chocolate Bar,Zero Non-Chlorine Bleach,snacks,1
4048215,Puffs Simply White Cheddar Cheese Flavored Snacks,Classic Hummus Family Size,snacks,1
4048216,Puffs Simply White Cheddar Cheese Flavored Snacks,Original Powder Coffee Creamer,snacks,1
4048217,Wavy Ranch Potato Chips,Beef Summer Sausage,snacks,1


## Customer Retention and Churn
#### Track customers' purchase frequency over time, helping to identify customers who haven't purchased recently (potential churn) or those who consistently buy (loyal customers).
 Customers classified as "Loyal" are making purchases at or above their typical frequency, while those classified as "Churned" are exhibiting longer-than-usual gaps between orders.
For "Loyal" customers: Reward them with loyalty programs, discounts, or exclusive offers to maintain engagement.
For "Churned" customers: Send targeted re-engagement campaigns (e.g., reminders, discounts, or personalized offers) to bring them back.


In [ ]:
query = """
    WITH avg_days AS (
        SELECT
            f.user_id,
            AVG(o.days_since_prior_order) AS avg_days
        FROM
            orders o
        JOIN
            fact f ON o.order_id = f.order_id
        WHERE
            f.user_id IS NOT NULL
        GROUP BY
            f.user_id
    )

    SELECT
        Distinct f.user_id,
        o.order_id,
        o.days_since_prior_order,
        avg_days.avg_days,
        CASE
            WHEN o.days_since_prior_order > avg_days.avg_days THEN 'Churned'
            ELSE 'Loyal'
        END AS customer_status
    FROM
        orders o
    JOIN
        fact f ON o.order_id = f.order_id
    JOIN
        avg_days ON f.user_id = avg_days.user_id
    WHERE
        f.user_id IS NOT NULL
    ORDER BY
        f.user_id, o.order_id;


    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,user_id,order_id,days_since_prior_order,avg_days,customer_status
0,4,94891,15.0,15.000000,Loyal
1,7,23391,28.0,28.000000,Loyal
2,10,68288,30.0,23.684211,Churned
3,10,83395,14.0,23.684211,Loyal
4,13,19256,9.0,9.000000,Loyal
...,...,...,...,...,...
99255,206206,35883,2.0,1.727273,Churned
99256,206206,68751,3.0,1.727273,Churned
99257,206207,9620,8.0,21.619048,Loyal
99258,206207,85132,30.0,21.619048,Churned


# Product Sales Trends:
### cumulative sales to see how sales grow or decline for different product categories.
### Rate of accumulation of each product according the number of orders (Orders Accumulation)


In [ ]:
query = """
SELECT
    p.department,
    p.product_name,
    SUM(f.quantity * p.unit_price) OVER (PARTITION BY p.product_name ORDER BY o.order_id) AS cumulative_sales
FROM
    fact f
JOIN
    products p ON f.product_id = p.product_id
JOIN
    orders o ON f.order_id = o.order_id;
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)


,department,product_name,cumulative_sales
0,beverages,#2 Coffee Filters,22.284178
1,beverages,#2 Coffee Filters,44.568357
2,beverages,#2 Coffee Filters,59.424475
3,beverages,#2 Coffee Filters,81.708654
4,beverages,#2 Coffee Filters,111.420891
...,...,...,...
999995,snacks,with Xylitol Unwrapped Spearmint 50 Sticks Sug...,5.384225
999996,snacks,with Xylitol Watermelon Twist 18 Sticks Sugar ...,2.547426
999997,snacks,with Xylitol Watermelon Twist 18 Sticks Sugar ...,5.094853
999998,beverages,with a Splash of Mango Coconut Water,6.169756


 # Identifying Top Customers by Revenue:
 Top 5 Customers:
User ID 128627: $5,601.63 (Rank 1)

User ID 145686: $4,793.86 (Rank 2)

User ID 169550: $4,292.78 (Rank 3)

User ID 2335: $3,992.93 (Rank 4)

User ID 116698: $3,969.67 (Rank 5)


Insight: These customers are the most valuable and should be prioritized for retention and loyalty programs.


A large number of customers (67,031 to 67,035) have $0.00 spending, indicating inactivity or low engagement.

In [ ]:
query = """
SELECT
    user_id,
    SUM(f.quantity * p.unit_price) AS total_spent,
    RANK() OVER (ORDER BY SUM(f.quantity * p.unit_price) DESC) AS revenue_rank
FROM
    fact f
JOIN
    products p ON f.product_id = p.product_id
GROUP BY
    user_id
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)


,user_id,total_spent,revenue_rank
0,128627,5601.629775,1
1,145686,4793.863978,2
2,169550,4292.777328,3
3,2335,3992.931714,4
4,116698,3969.669478,5
...,...,...,...
67031,114905,1.224860,67032
67032,165936,0.000000,67033
67033,104845,0.000000,67033
67034,97384,0.000000,67033


## To show the number of times product pairs are purchased together and their contribution to revenue.

The output reveals the most frequently purchased product pairs along with their revenue generation.The top pairs, such as Organic Strawberries and Banana, contribute significantly to revenue, with Organic Bananas and Organic Strawberries showing high purchase frequencies and substantial revenue generation. This insight indicates strong demand for these paired products, making them ideal candidates for strategic store placements, such as positioning these products close to each other or promoting them as bundles.
The rows with a revenue of 0 for some product pairs, suggest that these products are rarely purchased together, and may not have sufficient sales data. These can be considered for further analysis to explore potential marketing opportunities for underperforming products.

In [ ]:
query = """
    WITH ProductPairs AS (
        SELECT
            f1.product_id AS product_id_1,
            f2.product_id AS product_id_2,
            p1.department,
            COUNT(*) AS pair_count,
            SUM(f1.quantity * p1.unit_price) + SUM(f2.quantity * p2.unit_price) AS pair_revenue
        FROM
            fact f1
        JOIN
            fact f2 ON f1.order_id = f2.order_id
        JOIN
            products p1 ON f1.product_id = p1.product_id
        JOIN
            products p2 ON f2.product_id = p2.product_id
        WHERE
            f1.product_id < f2.product_id  -- To avoid duplicate pairs
        GROUP BY
            f1.product_id, f2.product_id, p1.department
    )
    SELECT
        p1.product_name AS product_1,
        p2.product_name AS product_2,
        pp.department,
        pp.pair_count AS times_purchased_together,
        pp.pair_revenue AS revenue_generated
    FROM
        ProductPairs pp
    JOIN
        products p1 ON pp.product_id_1 = p1.product_id
    JOIN
        products p2 ON pp.product_id_2 = p2.product_id
    ORDER BY
        pp.pair_revenue DESC;  -- Sort by the revenue generated from pairs


    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,product_1,product_2,department,times_purchased_together,revenue_generated
0,Organic Strawberries,Banana,produce,1786,135718.059084
1,Bag of Organic Bananas,Organic Strawberries,produce,1875,133952.967415
2,Bag of Organic Bananas,Organic Hass Avocado,produce,1896,120235.245039
3,Banana,Organic Avocado,produce,1647,106267.866748
4,Organic Baby Spinach,Banana,produce,1616,89190.834905
...,...,...,...,...,...
4048214,Organic Audacia Granola,Organic Ground Chicken,missing,1,0.000000
4048215,Mixed Berry Nonfat Yogurt Drink,"Oikos Yogurt Drink, Berry",missing,1,0.000000
4048216,Mixed Berry Nonfat Yogurt Drink,Vanilla Drink Crafted With Authentic Greek Yogurt,missing,1,0.000000
4048217,"Oikos Yogurt Drink, Berry",Vanilla Drink Crafted With Authentic Greek Yogurt,missing,1,0.000000


## Product Sales Trends:


In [ ]:
query = """
    SELECT
        p.department,
        p.product_name,
        SUM(f.quantity * p.unit_price) OVER (PARTITION BY p.department ORDER BY o.days_since_prior_order) AS cumulative_sales
    FROM
        fact f
    JOIN
        products p ON f.product_id = p.product_id
    JOIN
        orders o ON f.order_id = o.order_id
    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,department,product_name,cumulative_sales
0,alcohol,Pinot Grigio,1.901114e+04
1,alcohol,Belgian White,1.901114e+04
2,alcohol,Sonoma Pinot Noir Wine,1.901114e+04
3,alcohol,Sonoma Pinot Noir Wine,1.901114e+04
4,alcohol,80 Vodka Holiday Edition,1.901114e+04
...,...,...,...
999995,snacks,Olive Oil Sea Salt Bar,1.333574e+06
999996,snacks,Salted Peanut Butter Filled Pretzels,1.333574e+06
999997,snacks,Berry Bounty Trail Mix,1.333574e+06
999998,snacks,Chopped Walnuts,1.333574e+06


## Identifying Top Customers by Revenue:
Top 5 Customers:
User ID 128627: $5,601.63 (Rank 1)

User ID 145686: $4,793.86 (Rank 2)

User ID 169550: $4,292.78 (Rank 3)

User ID 2335: $3,992.93 (Rank 4)

User ID 116698: $3,969.67 (Rank 5)


Insight: These customers are the most valuable and should be prioritized for retention and loyalty programs.


A large number of customers (67,031 to 67,035) have $0.00 spending, indicating inactivity or low engagement.


In [ ]:
query = """
    SELECT
        user_id,
        SUM(f.quantity * p.unit_price) AS total_spent,
        RANK() OVER (ORDER BY SUM(f.quantity * p.unit_price) DESC) AS revenue_rank
    FROM
        fact f
    JOIN
        products p ON f.product_id = p.product_id
    GROUP BY
        user_id

    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,user_id,total_spent,revenue_rank
0,128627,5601.629775,1
1,145686,4793.863978,2
2,169550,4292.777328,3
3,2335,3992.931714,4
4,116698,3969.669478,5
...,...,...,...
67031,114905,1.224860,67032
67032,165936,0.000000,67033
67033,104845,0.000000,67033
67034,97384,0.000000,67033


## To rank each product within its category (department)
insights: we get the Top-Selling Products by Department(best-selling products within each department.)


Alcohol Category: A few top-selling products (especially wines) drive most of the revenue.

 Focus on retaining and promoting these products while exploring ways to boost sales of other alcohol types.


Snacks Category: Many products sell very little, indicating a need for better differentiation and marketing.

 Focus on improving the appeal of low-performing snacks or reducing the product range to focus on bestsellers.

In [ ]:
query = """
    SELECT
        p.department AS category,
        p.product_name,
        COUNT(*) AS total_sales,
    DENSE_RANK() OVER (PARTITION BY p.department ORDER BY COUNT(*) DESC) AS product_rank_within_category
    FROM
        fact f
    JOIN
        products p ON f.product_id = p.product_id
    GROUP BY
        p.department, p.product_name
    ORDER BY
        p.department, product_rank_within_category;

    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,category,product_name,total_sales,product_rank_within_category
0,alcohol,Sauvignon Blanc,239,1
1,alcohol,Chardonnay,183,2
2,alcohol,Cabernet Sauvignon,182,3
3,alcohol,Vodka,171,4
4,alcohol,Beer,163,5
...,...,...,...,...
35093,snacks,"100% Fruit Bar, Blueberry Pomegranate",1,230
35094,snacks,100 Calorie Variety Sack,1,230
35095,snacks,100 Calorie Vanilla Roast Almonds,1,230
35096,snacks,100 Calorie Right Bites Crackers,1,230


### Comparing products within departments based on revenue and identifying the top-performing and lowest-performing products in each department.

Alcohol Category: A few products (e.g., Sauvignon Blanc) dominate revenue, while others (e.g., Variety of Victorys) underperform.


Snacks Category: Some products (e.g., Lightly Salted Baked Snap Pea Crisps) are highly successful, while others (e.g., Turkey Jerky) contribute minimally.


In [ ]:
query = """
    SELECT
        p.department AS category,
        p.product_name,
        SUM(o.Revenue) AS total_revenue_per_product,  -- Calculate the total revenue for each product
        FIRST_VALUE(p.product_name) OVER (PARTITION BY p.department ORDER BY SUM(o.Revenue)) AS least_product_within_category,   -- Product with the least revenue
        FIRST_VALUE(p.product_name) OVER (PARTITION BY p.department ORDER BY SUM(o.Revenue) DESC) AS highest_product_within_category  -- Product with the highest revenue
    FROM
        fact f
    JOIN
        products p ON f.product_id = p.product_id
    JOIN
        orders o ON f.order_id = o.order_id
    GROUP BY
        p.department, p.product_name
    ORDER BY
        p.department;


    """
chunk_result = process_chunk(chunk, query)

display(chunk_result)

,category,product_name,total_revenue_per_product,least_product_within_category,highest_product_within_category
0,alcohol,Variety of Victorys,10.274430,Variety of Victorys,Sauvignon Blanc
1,alcohol,Bud Light Glass Bottles Beer,22.446243,Variety of Victorys,Sauvignon Blanc
2,alcohol,Extra Dry Gin,22.732067,Variety of Victorys,Sauvignon Blanc
3,alcohol,1500 Pale Ale,23.759419,Variety of Victorys,Sauvignon Blanc
4,alcohol,The Big Kahuna Variety Pack,30.497343,Variety of Victorys,Sauvignon Blanc
...,...,...,...,...,...
35093,snacks,Crackers Cheddar Bunnies Snack Packs,132478.649106,Turkey Jerky,Lightly Salted Baked Snap Pea Crisps
35094,snacks,Pretzel Crisps Original Deli Style Pretzel Cra...,144960.581467,Turkey Jerky,Lightly Salted Baked Snap Pea Crisps
35095,snacks,Sea Salt Pita Chips,147683.600000,Turkey Jerky,Lightly Salted Baked Snap Pea Crisps
35096,snacks,Original Veggie Straws,160699.335186,Turkey Jerky,Lightly Salted Baked Snap Pea Crisps


In [ ]:
# Sort the final results by total revenue
final_result = aisle_revenue.sort_values(by="total_revenue", ascending=False)

# Display the result
print(final_result)


                          aisle  total_revenue
50                 fresh fruits   1.051195e+08
53             fresh vegetables   4.719552e+07
133                      yogurt   3.591340e+07
4             baby food formula   3.282699e+07
98   packaged vegetables fruits   2.227257e+07
..                          ...            ...
8                        beauty   1.046235e+05
44                 eye ear care   9.956349e+04
84                     mint gum   6.142778e+04
58                 frozen juice   5.690779e+04
85                      missing   0.000000e+00

[134 rows x 2 columns]
